In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from tkinter import Tk
from collections import Counter

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# Visit the LegenVD's deck list site
url = 'https://aetherhub.com/User/LegenVD/Decks'
browser.visit(url)

In [ ]:
# Create a Beautiful Soup object
html = browser.html
decks_soup = soup(html, 'html.parser')

In [ ]:
# Get the names of all the decks in the first page
table = decks_soup.find('table', id='metaHubTable')
table_body = table.find('tbody')
table_rows = table_body.find_all('tr')
# table_titles = table_rows[0].find_all('a')
# table_titles[0].text

for i in table_rows:
    table_titles = i.find_all('a')
    table_href = i.find_all('a', href=True)
    print(table_titles[0].text)
    print(table_href[0]['href'])
    print('-------------------')
    

In [ ]:
# Find the next page button and click it
browser.links.find_by_partial_text('>').click()

In [ ]:
# Create a for loop to go through all the pages and get the names and links of all the decks
for i in range(0, 1):
    html = browser.html
    table = decks_soup.find('table', id='metaHubTable')
    table_body = table.find('tbody')
    table_rows = table_body.find_all('tr')
    time.sleep(5)
    for i in table_rows:
        table_titles = i.find_all('a')
        table_href = i.find_all('a', href=True)
        print(table_titles[0].text)
        print(table_href[0]['href'])
        print('-------------------')
    time.sleep(5)
    browser.links.find_by_partial_text('>').click()

In [ ]:
# Do the same as above but update the decks_soup variable after each click
for i in range(0, 2):
    html = browser.html
    decks_soup = soup(html, 'html.parser')
    table = decks_soup.find('table', id='metaHubTable')
    table_body = table.find('tbody')
    table_rows = table_body.find_all('tr')
    time.sleep(5)
    for i in table_rows:
        table_titles = i.find_all('a')
        table_href = i.find_all('a', href=True)
        print(table_titles[0].text)
        print(table_href[0]['href'])
        print('-------------------')
    time.sleep(5)
    browser.links.find_by_partial_text('>').click()
    decks_soup = soup(html, 'html.parser')

# Start here

In [ ]:
# Create a function that does the above and adds the information to a dictionary
# And saves the dictionary to a csv file
def scrape_deck_info(deck_info, url, end_range):
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.visit(url)
    partial_url = 'https://aetherhub.com'
    for i in range(0, end_range):
        html = browser.html
        decks_soup = soup(html, 'html.parser')
        table = decks_soup.find('table', id='metaHubTable')
        time.sleep(5)
        table_body = table.find('tbody')
        time.sleep(5)
        table_rows = table_body.find_all('tr')
        time.sleep(5)
        for i in table_rows:
            table_titles = i.find_all('a')
            table_href = i.find_all('a', href=True)
            # Find text with class 'text-muted'
            table_format = i.find_all('span', class_='text-muted')
            deck_info.append({'name': table_titles[0].text, 'link': partial_url + table_href[0]['href'], 'format': table_format[0].text})
            # Check if there are duplicates in the link key inside the list of dictionaries
            # If there are, stop the loop
            if len(Counter([i['link'] for i in deck_info])) < len(deck_info):
                break    
        time.sleep(5)
        # Click the '>' button, except for the last page, then stop the loop
        try:
            browser.links.find_by_partial_text('>').click()
        except:
            break
        decks_soup = soup(html, 'html.parser')
    deck_info_df = pd.DataFrame(deck_info)
    # Remove quotes from the name column
    deck_info_df['name'] = deck_info_df['name'].str.replace('"', '')
    deck_info_df.to_csv('deck_info.csv', index=False)
    return deck_info[0:5]

In [ ]:
deck_info = []
scrape_deck_info(deck_info, "https://aetherhub.com/User/LegenVD/Decks", 50)

## Scraping the decklist

In [2]:
# Import the csv file as a dataframe
deck_info_df = pd.read_csv('deck_info.csv')
deck_info_df.head()

,name,link,format
0,Training Grounds Transformers,https://aetherhub.com/Deck/training-grounds-tr...,Arena Standard
1,Samut Haste Aggro,https://aetherhub.com/Deck/samut-haste-aggro,Arena Standard
2,Multiverse Mill,https://aetherhub.com/Deck/multiverse-mill-912521,Arena Standard
3,Izzet Invasion,https://aetherhub.com/Deck/izzet-invasion,Arena Standard
4,Boros Flicker,https://aetherhub.com/Deck/boros-flicker-908904,Arena Standard


In [ ]:
# Loop through the links to get the deck lists and save them as text files
for i in deck_info_df['link'][0:2]:
    browser.visit(i)
    browser.links.find_by_partial_text('Export').click()
    browser.links.find_by_partial_text('Text List').click()
    # Click button with id "exportListbtn"
    browser.find_by_id('exportListbtn').click()
    text = Tk().clipboard_get()
    text = text.split('\n')
    # Make the name of the file based on the 'name' column
    file_name = i.split('/')[-1]
    file_name = file_name.replace('-', '_')
    file_name = file_name.replace(' ', '_')
    with open(f'deck_lists/{file_name}.txt', 'w') as f:
        for i in text:
            f.write(i + '\n')

In [3]:
# Create a function that does the above
def scrape_deck_lists(deck_info_df, start_range: int):
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    for i in deck_info_df['link'][start_range:]:
        time.sleep(3)
        browser.visit(i)
        browser.links.find_by_partial_text('Export').click()
        time.sleep(3)
        try:
            browser.find_by_css('a[href="#importModal"]').click()
        except:
            browser.links.find_by_partial_text('Text List').click()
        time.sleep(3)
        # Click button with id "exportListbtn"
        browser.find_by_id('exportListbtn').click()
        time.sleep(1)
        text = Tk().clipboard_get()
        text = text.split('\n')
        # Make the name of the file based on the 'name' column
        file_name = i.split('/')[-1]
        file_name = file_name.replace('-', '_')
        file_name = file_name.replace(' ', '_')
        with open(f'deck_lists/{file_name}.txt', 'w') as f:
            for i in text:
                f.write(i + '\n')
    return None

In [4]:
scrape_deck_lists(deck_info_df, 774)

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:00<00:00, 19.7MB/s]
